In [1]:
# simple RNN实线IMDB电影评论分类

In [2]:
from tensorflow.keras.datasets import imdb

In [3]:
# 加载数据


# 设置词典长度为常用的10000个单词，其他的是OOV（超出词汇）
vocabulary = 10000

# 开始
start_char = 1
oov_char = 2
index_from = 3

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary,start_char=start_char, oov_char=oov_char, index_from=index_from)

C:\conda\envs\course_video_cls\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\conda\envs\course_video_cls\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
# 打印第一个句子的长度
len(x_train[0])

218

In [5]:
# 打印第一个句子
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [6]:
# 索引和标签词典

In [7]:
word_index = imdb.get_word_index()

In [8]:
word_index

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [9]:
# 遍历查看
for key,val in word_index.items():
    print(key,val)
    break

fawn 34701


In [10]:
# 交换key和val
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

In [11]:
inverted_word_index[40834]

"harpo's"

In [12]:
# 解析第一个句子
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"


decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

In [13]:
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert [OOV] is an amazing actor and now the same being director [OOV] father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for [OOV] and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also [OOV] to the two little boy's that played the [OOV] of norman and paul they were just brilliant children are often left out of the [OOV] list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [14]:
# 统计长度
len_list = []
for i in x_train:
    len_list.append(len(i))

In [15]:
import numpy as np

In [16]:
# 查看平均长度
np.array(len_list).mean()

238.71364

In [17]:
# 补白
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
# 最大长度
word_num = 250
x_train = pad_sequences(x_train, maxlen=word_num)
x_test = pad_sequences(x_test, maxlen=word_num)

In [19]:
x_train.shape,x_test.shape

((25000, 250), (25000, 250))

In [20]:
# 搭建模型

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,SimpleRNN,Activation


In [22]:
x_train.shape

(25000, 250)

In [23]:
embed_dim = 32
state_dim = 32 

In [24]:
# 定义Many-to-one simple RNN模型
rnn = Sequential([
    # 输入大小
    Embedding(input_dim=vocabulary,output_dim=embed_dim,input_length=word_num),
    # 只需要最后一个状态向量
    SimpleRNN(state_dim, return_sequences=False),
    Dense(1,activation='sigmoid')
])


In [25]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 32)           320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [27]:
# 编译模型
rnn.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [28]:
# 训练
rnn.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Epoch 1/5
196/196 [==============================] - 27s 137ms/step - loss: 0.6327 - accuracy: 0.6255 - val_loss: 0.5583 - val_accuracy: 0.7088
Epoch 2/5
196/196 [==============================] - 27s 138ms/step - loss: 0.3918 - accuracy: 0.8325 - val_loss: 0.3916 - val_accuracy: 0.8288
Epoch 3/5
196/196 [==============================] - 28s 141ms/step - loss: 0.2903 - accuracy: 0.8836 - val_loss: 0.3574 - val_accuracy: 0.8567
Epoch 4/5
196/196 [==============================] - 27s 137ms/step - loss: 0.2231 - accuracy: 0.9133 - val_loss: 0.3871 - val_accuracy: 0.8387
Epoch 5/5
196/196 [==============================] - 28s 141ms/step - loss: 0.1607 - accuracy: 0.9406 - val_loss: 0.8569 - val_accuracy: 0.7450


In [29]:
score, acc = rnn.evaluate(x_test, y_test, batch_size=128)


196/196 [==============================] - 3s 14ms/step - loss: 0.8569 - accuracy: 0.7450


In [30]:
from tensorflow.keras.layers import Flatten

In [31]:
# 定义Many-to-many模型
rnn_new = Sequential([
    # Flatten大小
    Embedding(input_dim=vocabulary,output_dim=embed_dim,input_length=word_num),
    # 输出所有的状态向量，sequence（矩阵）
    SimpleRNN(state_dim, return_sequences=True),
    # 拉直
    Flatten(),
    Dense(1,activation='sigmoid')
])


In [32]:
rnn_new.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 32)           320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 250, 32)           2080      
_________________________________________________________________
flatten (Flatten)            (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8001      
Total params: 330,081
Trainable params: 330,081
Non-trainable params: 0
_________________________________________________________________


In [33]:
rnn_new.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [34]:
rnn_new.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Epoch 1/5
196/196 [==============================] - 27s 136ms/step - loss: 0.4960 - accuracy: 0.7358 - val_loss: 0.5345 - val_accuracy: 0.7573
Epoch 2/5
196/196 [==============================] - 28s 142ms/step - loss: 0.2560 - accuracy: 0.8942 - val_loss: 0.3146 - val_accuracy: 0.8667
Epoch 3/5
196/196 [==============================] - 27s 135ms/step - loss: 0.1538 - accuracy: 0.9428 - val_loss: 0.4005 - val_accuracy: 0.8398
Epoch 4/5
196/196 [==============================] - 26s 134ms/step - loss: 0.0696 - accuracy: 0.9764 - val_loss: 0.6036 - val_accuracy: 0.8032
Epoch 5/5
196/196 [==============================] - 27s 137ms/step - loss: 0.0275 - accuracy: 0.9917 - val_loss: 0.6492 - val_accuracy: 0.8432
